In [913]:
import numpy as np
import heapq

In [914]:
def computeMinPath(graph, numVertices):
  minPath = np.zeros((numVertices, 2))
  for i in range(numVertices):
    edgeWeights = graph[i]
    
    minEdge = np.min(edgeWeights)
    secondMinEdge = np.partition(edgeWeights, 1)[1]

    minPath[i] = [minEdge, secondMinEdge]

  return minPath
      
def getOnePath(graph, numVertices):
  path = np.array([0])
  cost = 0
  for i in range(1, numVertices):
    cost += graph[path[-1]][i]
    path = np.concatenate([path, [i]]).astype(int)
  cost += graph[path[-1]][0]
  path = np.concatenate([path, [0]]).astype(int)
  return path, cost

class Node:
  def __init__(self, graph, minPath, prevNode=None, nextVertice=None):
    if prevNode is None or nextVertice is None:
      self.level = 1
      self.path = [0]
      self.cost = 0
      self.bound = bound(graph, minPath, 0)
      return

    self.level = prevNode.level + 1
    self.path = np.concatenate([prevNode.path, [nextVertice]]).astype(int)
    if len(prevNode.path) == 0:
      self.cost = 0
    else:
      self.cost = prevNode.cost + graph[prevNode.path[-1]][nextVertice]
    self.bound = bound(graph, minPath, nextVertice, prevNode)

  def __lt__(self, other):
    return self.bound < other.bound
  
  def __gt__(self, other):
    return self.bound > other.bound
  
  def __eq__(self, other):
    return self.bound == other.bound
  
  def __str__(self) -> str:
    return f"Path: {self.path} Cost: {self.cost} Bound: {self.bound}"


def bound(graph, minPath, nextVertice, prevNode=None):
  if prevNode is None:
    return np.sum(minPath) / 2
  
  cost = graph[prevNode.path[-1]][nextVertice]

  bound = prevNode.bound * 2

  if cost >= minPath[nextVertice][1]:
    bound -= minPath[nextVertice][1]
    bound += cost
  
  if cost >= minPath[prevNode.path[-1]][1]:
    bound -= minPath[prevNode.path[-1]][1]
    bound += cost

  return bound / 2


In [915]:
def branchAndBoundTSP(graph, numVertices):
  minPath = computeMinPath(graph, numVertices)

  root = Node(graph, minPath, None, 0)
  queue = [root]
  heapq.heapify(queue)

  solution, best = getOnePath(graph, numVertices)

  while queue:
    node = heapq.heappop(queue)
    
    if node.level >= numVertices:
      finalNode = Node(graph, minPath, node, 0)
      if finalNode.cost < best:
        best = finalNode.cost
        solution = finalNode.path
    
    elif node.bound < best:
      unvisited = np.setdiff1d(np.arange(numVertices), node.path)
      for k in unvisited:
        newNode = Node(graph, minPath, node, k)
        if newNode.bound < best and graph[node.path[-1]][k] != np.inf:
          heapq.heappush(queue, newNode)

  return solution, best

In [916]:
def generateMST(graph, numVertices):
  included = [False] * numVertices
  included[0] = True

  cost = 0
  edges = []
  numVisited = 1

  while numVisited < numVertices:
    minEdge = (float('inf'), None, None) 

    for i in range(numVertices):
      if included[i]:
        for j in range(numVertices):
          if not included[j] and graph[i][j] < minEdge[0]:
            minEdge = (graph[i][j], i, j)

    cost, startVertex, endVertex = minEdge
    included[endVertex] = True
    cost += cost
    edges.append((startVertex, endVertex))
    numVisited += 1

  mstAdjMatrix = np.full((numVertices, numVertices), np.inf)
  for edge in edges:
    mstAdjMatrix[edge[0]][edge[1]] = mstAdjMatrix[edge[1]][edge[0]] = graph[edge[0]][edge[1]]

  return mstAdjMatrix, cost

In [917]:
def preorderWalkAux(adjMatrix, startVertex, visited, order):
  order.append(startVertex)
  visited[startVertex] = True

  for neighbor in range(len(adjMatrix)):
    if adjMatrix[startVertex][neighbor] != np.inf and not visited[neighbor]:
      preorderWalkAux(adjMatrix, neighbor, visited, order)

def preorderWalk(adjMatrix, start_vertex=0):
  numVertices = len(adjMatrix)
  visited = [False] * numVertices
  order = []

  preorderWalkAux(adjMatrix, start_vertex, visited, order)

  return order

In [918]:
def approximateTwiceAroundTheTree(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  preorder = preorderWalk(mstAdjMatrix)
  tour = preorder + [preorder[0]]

  tourCost = sum(graph[tour[i]][tour[i + 1]] for i in range(len(tour) - 1))

  return tour, tourCost

In [919]:
def findMinimumWeightMatching(graph, oddVertices):
  if oddVertices is None:
    oddVertices = range(len(graph))

  edges = []
  for i in oddVertices:
    for j in oddVertices:
      if i < j and graph[i][j] != np.inf:
        edges.append((i, j, graph[i][j]))

  edges.sort(key=lambda x: x[2])

  matching = []
  visited = np.zeros(len(graph), dtype=bool)

  for edge in edges:
    if not visited[edge[0]] and not visited[edge[1]]:
      matching.append((edge[0], edge[1]))
      visited[edge[0]] = True
      visited[edge[1]] = True

  return matching

In [920]:
def getEulerianCircuit(graph):
  startVertex = 0  # Choose a starting vertex
  numVertices = len(graph)
  visited = np.zeros(numVertices, dtype=bool)
  tour = []
  stack = np.array([startVertex])

  while stack.size > 0:
    vertex = stack[-1]
    if not visited[vertex]:
      visited[vertex] = True
      tour.append(vertex)
    unvisited_neighbors = np.where((graph[vertex] != np.inf) & (~visited))[0]
    if unvisited_neighbors.size > 0:
      next_vertex = unvisited_neighbors[0]
      stack = np.append(stack, next_vertex)
    else:
      stack = stack[:-1]

  return tour

In [921]:
def christofides(graph, numVertices):
  mstAdjMatrix, mstCost = generateMST(graph, numVertices)

  oddVertices = [vertex for vertex, degree in enumerate(np.sum(mstAdjMatrix != np.inf, axis=1)) if degree % 2 != 0]

  matchingEdges = findMinimumWeightMatching(graph, oddVertices)

  multigraph = mstAdjMatrix.copy()
  for edge in matchingEdges:
    multigraph[edge[0]][edge[1]] = multigraph[edge[1]][edge[0]] = graph[edge[0]][edge[1]]

  eulerianCircuit = getEulerianCircuit(multigraph)
  hamiltonianCircuit = list(dict.fromkeys(eulerianCircuit))
  hamiltonianCircuit.append(hamiltonianCircuit[0])

  tourCost = sum(graph[hamiltonianCircuit[i]][hamiltonianCircuit[i + 1]] for i in range(len(hamiltonianCircuit) - 1))

  return hamiltonianCircuit, tourCost

In [922]:
numVertices = 4
graph = np.full((numVertices, numVertices), np.inf)
graph = np.array([
       [0, 10, 15, 20],
       [10, 0, 35, 25],
       [15, 35, 0, 30],
       [20, 25, 30, 0]
       ])
solution, best = branchAndBoundTSP(graph, numVertices)

print(f"Solution: {solution} Cost: {best}")

Solution: [0 1 3 2 0] Cost: 80


In [923]:
christofides(graph, numVertices)

([0, 1, 2, 3, 0], 95)

In [924]:
def preorder_walk(graph, start_vertex, visited):
  print(start_vertex, end=" ")
  visited[start_vertex] = True

  for neighbor in range(len(graph)):
    if graph[start_vertex][neighbor] != np.inf and not visited[neighbor]:
      preorder_walk(graph, neighbor, visited)

In [925]:
class Point:
  def __init__(self, id, x, y):
    self.id = id
    self.x = x
    self.y = y

  def distance(self, point):
    return np.sqrt((self.x - point.x)**2 + (self.y - point.y)**2)
  
  def __str__(self) -> str:
    return f'Point {self.id} ({self.x}, {self.y})'

In [926]:
def readFile(file_name):
    with open(file_name) as f:
        content = f.readlines()
    content = [x.strip() for x in content if x[0].isdigit()]
    return content


In [927]:
content = readFile('ulysses22.tsp')

points = []
for line in content:
  id, x, y = line.split(' ')
  points.append(Point(int(id)-1, float(x), float(y)))

In [928]:
numVertices = len(points)
graph = np.full((numVertices, numVertices), np.inf)

for p in points:
  for q in points:
    if p.id != q.id:
      graph[p.id][q.id] = p.distance(q)

print(graph)


[[        inf  5.88232947  5.42147581  3.34819354 10.96685917  8.25804456
   7.31221581  0.72027772 11.70822361  7.9310655  25.7208573   5.29499764
   5.07079875  5.30050941  6.6912256   1.41209065  3.9428543   3.35840736
   7.19461604  6.60916031  6.54622028  2.24243172]
 [ 5.88232947         inf  1.29189783  4.48742688 16.75590045 14.10396044
  13.09061114  6.06684432 17.13061879 13.19734822 31.55360043 11.07476411
  10.89295185 11.17157106 12.51380438  6.59334513  2.32260629  4.6939216
  12.61004758 12.02404258 11.94681548  4.10951335]
 [ 5.42147581  1.29189783         inf  4.83011387 16.38825189 13.46836664
  12.39611633  5.74943475 16.23383196 12.28515364 30.85694897 10.40212478
  10.25714385 10.59834893 12.11217982  5.88367232  2.65        5.03619896
  11.75061275 11.1723677  11.09162747  4.06911538]
 [ 3.34819354  4.48742688  4.83011387         inf 12.88350884 11.00703866
  10.2403955   2.96141858 14.87485462 11.20325845 28.33057183  8.29000603
   7.99656176  8.04767047  8.83362

In [929]:
# print('Branch and bound', branchAndBoundTSP(graph, numVertices))

In [930]:
print("Twice around the tree", approximateTwiceAroundTheTree(graph, numVertices))

Twice around the tree ([0, 7, 21, 3, 17, 16, 1, 2, 15, 12, 11, 6, 5, 18, 9, 8, 10, 19, 20, 13, 14, 4, 0], 88.04848334468727)


In [931]:
print("Christofides", christofides(graph, numVertices))

Christofides ([0, 7, 21, 3, 17, 16, 1, 2, 10, 8, 9, 18, 6, 5, 11, 12, 4, 14, 13, 15, 19, 20, 0], 98.28858285156915)
